# Desikan Atlas

The Desikan parcellation is a cortical atlas with 68 regions, evenly spread across the left and right hemispheres [1]. It was originally developed as a surface atlas, and is made available through Freesurfer. However, when performing volumetric analysis, such a surface atlas is not useable. To remedy this, we have converted the Desikan parcellation to a surface atlas. We have also added two regions - region numbers 1 and 36, which fill the white matter space near the corpus collosum in each hemisphere.

[1]: http://www.sciencedirect.com/science/article/pii/S1053811906000437

## Purpose of this notebook

The Desikan volumetric atlas was created several years ago, and as tools and pipelines have since changed, oversights in how this conversion process was performed have been detected. As such, there have been several iterations of trying to correct this atlas, some appearing successful in some ways but not in others, and now many copies exist with different properties. This notebook will investigate all known copies of the Desikan volumetric atlas, and evaluate which copy of the atlas - whether existing prior to this notebook or not - should be used in future.

### Step 1: Get summary information

#### Step 1.1: Load template & atlases

In [1]:
# Set list of filenames
MNI152_fname = './data/MNI152/MNI152_T1_1mm_brain_mask.nii.gz'
desikan_fnames = ['./data/desikan_labels/brainstore_MR_atlases/parcellations/desikan.nii.gz',
                  './data/desikan_labels/brainstore_MR_atlases/parcellations/ingested/desikan.nii.gz',
                  './data/desikan_labels/brainstore_MR_atlases/parcellations/raw/desikan.nii.gz',
                  './data/desikan_labels/brainstore_MR_atlases/parcellations/raw/desikan_correct.nii.gz',
                  './data/desikan_labels/brainstore_mrocp_data_share_atlases/labels/desikan.nii.gz']

# Load images
import nibabel as nb
MNI152 = nb.load(MNI152_fname)
desikans = {idx+1:nb.load(fnames) for idx, fnames in enumerate(desikan_fnames)}

# Print reference info
print("desikans")
print("  "+"\n  ".join([str(idx+1)+': '+fnames for idx, fnames in enumerate(desikan_fnames)]))

desikans
  1: ./data/desikan_labels/brainstore_MR_atlases/parcellations/desikan.nii.gz
  2: ./data/desikan_labels/brainstore_MR_atlases/parcellations/ingested/desikan.nii.gz
  3: ./data/desikan_labels/brainstore_MR_atlases/parcellations/raw/desikan.nii.gz
  4: ./data/desikan_labels/brainstore_MR_atlases/parcellations/raw/desikan_correct.nii.gz
  5: ./data/desikan_labels/brainstore_mrocp_data_share_atlases/labels/desikan.nii.gz


#### Step 1.2: Get basic info about atlases

In [2]:
ims = {ky:desikans[ky].get_data() for ky in desikans.keys()}
# Get atlas shapes

shapes = {ky:ims[ky].shape for ky in ims.keys()}
print("shape")
print("  "+"\n  ".join([str(ky)+': '+str(shapes[ky]) for ky in shapes.keys()]))


import numpy as np

# Get number of atlas regions (+ background)
nrois = {ky:len(np.unique(ims[ky])) for ky in ims.keys()}
print("\nnrois")
print("  "+"\n  ".join([str(ky)+': '+str(nrois[ky]) for ky in nrois.keys()]))

# Get range of ROI values
roi_range = {ky:np.max(ims[ky])-np.min(ims[ky]) for ky in ims.keys()}
print("\nroi_range")
print("  "+"\n  ".join([str(ky)+': '+str(roi_range[ky]) for ky in roi_range.keys()]))

# Get ROI value datatypes
dtypes = {ky:ims[ky].dtype for ky in ims.keys()}
print("\nroi_range")
print("  "+"\n  ".join([str(ky)+': '+str(dtypes[ky]) for ky in dtypes.keys()]))

shape
  1: (182, 218, 182)
  2: (182, 218, 182)
  3: (182, 218, 182)
  4: (182, 218, 182)
  5: (182, 218, 182)

nrois
  1: 71
  2: 71
  3: 71
  4: 71
  5: 71

roi_range
  1: 1174405120
  2: 70
  3: 135
  4: 135
  5: 135

roi_range
  1: int32
  2: >i4
  3: >i4
  4: >i4
  5: >i4


#### Step 1.3:  Get affine transforms

In [3]:
print("\nMNI152 Affine")
print(MNI152.get_affine())

affines = {ky:desikans[ky].get_affine() for ky in desikans.keys()}
print("\naffines")
for idx, val in enumerate(affines.values()):
    print(str(idx+1)+":\n"),
    print(val)


MNI152 Affine
[[ -1.   0.   0.  90.]
 [  0.  -1.   0.  91.]
 [  0.   0.   1. -72.]
 [  0.   0.   0.   1.]]

affines
1:
[[  -1.    0.    0.   90.]
 [   0.    1.    0. -126.]
 [   0.    0.    1.  -72.]
 [   0.    0.    0.    1.]]
2:
[[ -1.   0.   0.  90.]
 [  0.  -1.   0.  91.]
 [  0.   0.   1. -72.]
 [  0.   0.   0.   1.]]
3:
[[ -1.   0.   0.  90.]
 [  0.  -1.   0.  91.]
 [  0.   0.   1. -72.]
 [  0.   0.   0.   1.]]
4:
[[ -1.   0.   0.  90.]
 [  0.  -1.   0.  91.]
 [  0.   0.   1. -72.]
 [  0.   0.   0.   1.]]
5:
[[ -1.   0.   0.  90.]
 [  0.  -1.   0.  91.]
 [  0.   0.   1. -72.]
 [  0.   0.   0.   1.]]


#### Step 1.4: Evaluate overlap with template

In [27]:
MNI152_im = MNI152.get_data()
print(218*182*218)
# np.linalg.norm(MNI152_im, )
for im in ims:
    im = ims[im]
    im = im > 0
    print(np.sum(np.abs(MNI152_im - im)))

8649368
47726083
92163
92163
92163
92163


## Early signs....

- `1` is the original atlas that was created with a poor header and doesn't align in voxel space
- `2` is an attempt to normalize this atlas unsuccessfully/align it in image space, but broke the alignment in voxel space
- `3-5` are the same as one another, with the true region labels, and align in both image and voxel space. I suspect these are the ones to use moving forward, with potential relabeling of region labels 101-135 to 36-70.